In [39]:
import pandas as pd
import os 
from pathlib import Path
from bertopic import BERTopic
import re 
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image, display
import plotly.io as pio

import pandas as pd
import os 
from pathlib import Path
from nltk.corpus import stopwords

from preprocessing.preprocessing_module import full_cleaning_pipeline


METAPROJECT_NAME = 'TopicModelling_META'
SUBPROJECT_NAME = 'TopMod_pipeline'



DATASET_NAME = "SensoryTool_CombinedData.csv"
HIGH_SENSORY = True #If False, corresponds to deep listening experience (no flicker)
CONDITION = 'highsensory' if HIGH_SENSORY else 'deeplistening'
print(f'Condition : "{CONDITION}"')




PROJDIR = os.path.expanduser(f"~/projects/{METAPROJECT_NAME}")
DATADIR = os.path.join(PROJDIR,f'DATA/{DATASET_NAME}')
CODEDIR = os.path.join(PROJDIR,f'{SUBPROJECT_NAME}')



%matplotlib inline

Condition : "highsensory"


Parameters for BERTopic

https://maartengr.github.io/BERTopic/getting_started/embeddings/embeddings.html

In [44]:
from transformers.pipelines import pipeline

#Parameters for Topic Modelling
remove_stopwords = True

embedding_model = "all-MiniLM-L6-v2" #which transformer-based model to use to create embeddings of the reports
# embedding_model = pipeline("feature-extraction", model="distilbert-base-cased")
# topic_model = BERTopic(embedding_model=embedding_model)
min_topic_size = 11 #sets the minimum number of reports that a topic should have
min_topic_size_range = range(15, 1, -1)  # Adjust the range as per your requirement

Import Data

In [45]:
# Raw text data
dataset = pd.read_csv(DATADIR)['reflection_answer']
reports = pd.DataFrame(dataset[dataset.notna() & (dataset != '')].reset_index(drop=True)) #delete empty rows
reports

/var/folders/1d/q_0fy53s0jdcsc1sb09wkntw00z7v7/T/ipykernel_71439/2631862404.py:2: DtypeWarning:

Columns (23,24,71,73,75,77,109,111,112,113,114,115,116,117,118,119,120,121,122,123,124,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,157,159,161,165,168,196,203,205) have mixed types. Specify dtype option on import or set low_memory=False.



reflection_answer
0    thecoloursmade patterns when i had my eyes clo...
1    Intense chaos. And then my mind checked out an...
2                                        good stuff.\n
3    a pattern of red and white lights thatflashed ...
4    i wentback to many hard and mostly beautiful m...
..                                                 ...
429                         travelling through space\n
430  i thought alot about rands journey into rhudia...
431  the  visual here doesnt look anything like whs...
432  pleasant but should i be seeim something  unex...
433  I saw past experiences and reflected on my cho...

[434 rows x 1 columns]

In [46]:
#Preprocessed text data
#check if preprocessed dataset already exist, and if not, create a preprocessed dataset and store it in the preprocessing folder

base_name, ext = os.path.splitext(DATASET_NAME)
new_path = f"{base_name}_{CONDITION}_preprocessed_stopwords{ext}" if remove_stopwords else f"{base_name}_{CONDITION}_preprocessed{ext}"
preproc_path = os.path.join(PROJDIR, f'DATA/preprocessed/{new_path}')


# Check if the preprocessed file already exists
if not os.path.exists(preproc_path):
    # If not, create it using the full_cleaning_pipeline function and save it
    reports = full_cleaning_pipeline(reports, 'reflection_answer',text_clean_flag=remove_stopwords)
    reports = reports.reset_index(drop=True)
    
    # Ensure the directory exists before saving the file
    os.makedirs(os.path.dirname(preproc_path), exist_ok=True)
    
    reports.to_csv(preproc_path, index=False)
else:
    reports = pd.read_csv(preproc_path)
    print(f"The preprocessed file already exists at: {preproc_path}")

data = reports['reflection_answer']

print('N={} reports (HighSensory = {})'.format(len(reports),HIGH_SENSORY))
reports.head()

The preprocessed file already exists at: /Users/rb666/projects/TopicModelling_META/DATA/preprocessed/SensoryTool_CombinedData_highsensory_preprocessed_stopwords.csv
N=396 reports (HighSensory = True)


reflection_answer
0  thecoloursmade patterns eyes close almost like...
1  Intense chaos And mind checked unconscious too...
2  pattern red white lights thatflashed andbecame...
3  wentback many hard mostly beautiful memories w...
4                                        Hope colour

## BERTopic

https://miro.medium.com/v2/resize:fit:4800/format:webp/0*DxAqCqx6dzg0KQlg.png

In [47]:
# finds the maximum min_topic_size that doesn't cause an error

def find_max_topics(data, embedding_model, min_topic_size_range):
    """ 
    Returns the maximum number of topics and the optimal minimum topic size found :
    - max_topics: Keeps track of the maximum number of topics found during the iterations.
    - optimal_min_topic_size: Stores the min_topic_size value that resulted in max_topics.
    """
    max_topics = 0
    optimal_min_topic_size = 0
    
    for min_size in min_topic_size_range:
        try:
            model = BERTopic(language="english", 
                             calculate_probabilities=True, 
                             verbose=True,
                             nr_topics='auto', # Automatically determines the number of topics.
                             embedding_model=embedding_model,
                             min_topic_size=min_size)
            
            topics, _ = model.fit_transform(data)
            num_topics = len(set(topics))  # Calculate the number of unique topics
            
            if num_topics > max_topics:
                max_topics = num_topics
                optimal_min_topic_size = min_size
                
            print(f"Min_topic_size: {min_size}, Num Topics: {num_topics}")
            
        except ValueError as e:
            print(f"Error with min_topic_size: {min_size}. Error message: {str(e)}")
            continue
        
    return max_topics, optimal_min_topic_size

if not min_topic_size:
    max_topics, optimal_min_topic_size = find_max_topics(data, embedding_model, min_topic_size_range)
    print(f"Maximum number of topics: {max_topics} found with min_topic_size: {optimal_min_topic_size}")
else:
    optimal_min_topic_size = min_topic_size


print(f"Setting min_topic_size = {optimal_min_topic_size}")


Setting min_topic_size = 11


### Training the BERT model

In [48]:
from bertopic import BERTopic

model = BERTopic(language="english", calculate_probabilities=True, verbose=True,nr_topics='auto',embedding_model=embedding_model,min_topic_size=optimal_min_topic_size) #The higher the value of min_topic_size, the more specific the topics will be
topics, probs = model.fit_transform(reports['reflection_answer']) #Fit model to data : topics are the topic labels for each document, probs are the probabilities of each topic for each document
model.visualize_topics() #Visualize topics


Batches:   0%|          | 0/13 [00:00<?, ?it/s]

2023-10-13 14:41:37,717 - BERTopic - Transformed documents to Embeddings
2023-10-13 14:41:38,782 - BERTopic - Reduced dimensionality
2023-10-13 14:41:38,794 - BERTopic - Clustered reduced embeddings
2023-10-13 14:41:38,811 - BERTopic - Reduced number of topics from 6 to 6


### Topic extraction and representation

In [49]:
topic_freq = model.get_topic_freq().head() #Get the frequency of each topic
freq = model.get_topic_info() #see the most representative terms for each topic
print("N number of topics: ",len(freq))
print(freq.head())


N number of topics:  6
   Topic  Count                               Name  \
0     -1    182              -1_felt_like_saw_time   
1      0     75          0_colours_like_saw_colour   
2      1     50   1_experience_peace_relaxation_ho   
3      2     37          2_music_felt_like_feeling   
4      3     28  3_thoughts_asleep_felt_experience   

                                      Representation  \
0  [felt, like, saw, time, experience, space, bod...   
1  [colours, like, saw, colour, one, patterns, sh...   
2  [experience, peace, relaxation, ho, un, an, pe...   
3  [music, felt, like, feeling, thinking, took, e...   
4  [thoughts, asleep, felt, experience, mind, thi...   

                                 Representative_Docs  
0  [saw lot animals ones used like like reptile t...  
1  [experienced range images chose many colours s...  
2  [deep relaxation like deep tissue massage mind...  
3  [Anxiety loneliness inner child screaming love...  
4  [felt sleepy bit anxious times due sp

## Prileminary visualisation

In [50]:
outputs_path = os.path.join(CODEDIR,'BERT_outputs')

# Save the barchart visualization to a file
fig = model.visualize_barchart(top_n_topics=10) #Visualize the top 10 topics
save_path_barchart = os.path.join(outputs_path,'BERT_barchart.png')
fig.write_image(save_path_barchart)

# # Import the Image and display functions from IPython
# from IPython.display import Image, display

# Display the saved image
display(Image(filename=save_path_barchart))

In [51]:
# Save the barchart visualization to a file
fig = model.visualize_documents(reports['reflection_answer'])
save_path_cloud = os.path.join(outputs_path,'BERT_cloud_{}.png'.format(CONDITION))
fig.write_image(save_path_cloud)

# # Import the Image and display functions from IPython
# from IPython.display import Image, display

# Display the saved image
display(Image(filename=save_path_cloud))

In [52]:
model.visualize_heatmap(top_n_topics=10) #Visualize the top 10 topics as a heatmap